In [29]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse
import re
import random
import datetime

# ３章 Webクローラを書く
全ての情報は１ページに格納されているのならいいのだが，複数のURLに情報がまたがっている場合はどうやって取得すればいいだろうか．<br>「六次の隔たり」というように，インターネットの海を掻きまわる前提でスクレイピングしつつも可能な限り６以内のリンクで情報を取得することを目標としていく．


### 3.1 単一ドメインの走査
まずは任意のURLからページのリンクのリストを抽出する．

In [3]:
html = urlopen('https://en.wikipedia.org/wiki/The_Beatles')
bs = BeautifulSoup(html, 'html.parser')

for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Featured_articles
/wiki/Wikipedia:Protection_policy#semi
/wiki/File:The_Beatles,_Part_1.ogg
#mw-head
#searchInput
/wiki/The_Beatles_(album)
/wiki/Beatles_(disambiguation)
/wiki/Beetle
/wiki/Fab_Four_(disambiguation)
/wiki/File:The_Fabs.JPG
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/Ringo_Starr
/wiki/George_Harrison
/wiki/Liverpool
/wiki/Rock_music
/wiki/Pop_music
/wiki/Beat_music
/wiki/Psychedelic_music
/wiki/Parlophone
/wiki/Apple_Records
/wiki/Capitol_Records
/wiki/The_Quarrymen
/wiki/Tony_Sheridan
/wiki/Billy_Preston
/wiki/Plastic_Ono_Band
https://thebeatles.com
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/George_Harrison
/wiki/Ringo_Starr
#Personnel
/wiki/Rock_music
/wiki/Liverpool
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/George_Harrison
/wiki/Ringo_Starr
/wiki/Cultural_impact_of_the_Beatles
#cite_note-FOOTNOTEHasted2017425-1
/wiki/Counterculture_of_the_1960s
/wiki/Popular_music
#cite_note-FOOTNOTEFrontani2007125-2
/wiki/Skiffle
/wiki/Beat_music
/wiki/Rock_and

ただし，Wikipediaとかには項目リンクの他にサイドバー，ヘッダー．フッターなどの他の項目でないリンクが結構多い．実際の分析の時にこれを区別してやる必要があった<br>
項目ページとその他のページを区別できる方法がないか探したところ，以下の情報を取得．
- divの中にあって，idがbodyContentに格納されている．
- URLにコロンが入っていない．
- URLが/wiki/から始まる．

この規則に基づいて，正規表現を用いて目的の項目リンクだけを取得するようにコードを改変する．

In [9]:
html = urlopen('https://en.wikipedia.org/wiki/The_Beatles')
bs = BeautifulSoup(html, 'html.parser')

# 正規表現を通すことで，セミコロンとかが付いているリンクを除外できた
for link in bs.find("div",{"id":"bodyContent"}).find_all("a",href=re.compile("^(/wiki)((?!:).)*$")):
    if "href" in link.attrs: # .attrsをつけることでhrefとかtitleとかの属性を確認することができる
        print(link.attrs['href'])


/wiki/The_Beatles_(album)
/wiki/Beatles_(disambiguation)
/wiki/Beetle
/wiki/Fab_Four_(disambiguation)
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/Ringo_Starr
/wiki/George_Harrison
/wiki/Liverpool
/wiki/Rock_music
/wiki/Pop_music
/wiki/Beat_music
/wiki/Psychedelic_music
/wiki/Parlophone
/wiki/Apple_Records
/wiki/Capitol_Records
/wiki/The_Quarrymen
/wiki/Tony_Sheridan
/wiki/Billy_Preston
/wiki/Plastic_Ono_Band
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/George_Harrison
/wiki/Ringo_Starr
/wiki/Rock_music
/wiki/Liverpool
/wiki/John_Lennon
/wiki/Paul_McCartney
/wiki/George_Harrison
/wiki/Ringo_Starr
/wiki/Cultural_impact_of_the_Beatles
/wiki/Counterculture_of_the_1960s
/wiki/Popular_music
/wiki/Skiffle
/wiki/Beat_music
/wiki/Rock_and_roll
/wiki/Classical_music
/wiki/Traditional_pop
/wiki/Sentimental_ballad
/wiki/Music_of_India
/wiki/Psychedelic_music
/wiki/Hard_rock
/wiki/Recording_practices_of_the_Beatles
/wiki/Baby_boomers
/wiki/Lennon%E2%80%93McCartney
/wiki/The_Quarrymen
/wiki/Hamburg

これを一般化してやることで，様々なケースに応用できるようになると考えられる．

In [25]:
random.seed(datetime.datetime.now()) # 時間という変数を用いたランダムシード

def get_links(article_url):
    html = urlopen(f'http://www.wikipedia.org{article_url}')
    bs = BeautifulSoup(html, 'html.parser')

    return bs.find('div',{'id':'bodyContent'}).find_all('a',href=re.compile("^(/wiki)((?!:).)*$"))

links = get_links('/wiki/The_Beatles')

while len(links) > 0:
    new_article = links[random.randint(0,len(links)-1)].attrs['href']
    print(new_article)
    links = get_links(new_article)

### 3.2 サイト全体のクローリング
各webサイトを再帰的に走査した時，リンクをクローリングして，そこで得られたリンクをまたクローリング...なんてやっていくと，無限に走査してしまってキリがない．<br>取得した内部リンクが重複している可能性についても考慮していく．

In [26]:
pages = set()

def add_links(page_url):
    global pages
    html = urlopen(f'http://en.wikipedia.org{page_url}')
    bs = BeautifulSoup(html, 'html.parser')
    # 今回は/wiki/を含む全てのリンクを探索することにした
    for link in bs.find_all("a", href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 重複していない未発見のページを格納
                new_page = link.attrs['href']
                print(new_page)
                pages.add(new_page)
                add_links(new_page) # 再帰によりクローリング

add_links('')

### 3.3 インターネットをクローリング
webクローリングをするということは，どこへでも辿り着いてしまうということを示す．エロサイトにつながるかもしれないし，ダークウェブに繋がってしまうかもしれない．<br>然るべき制約を設けなければ危険である．そのため，Webクローリングを行う場合は以下のことに気をつけなくてはならない．
- どういうデータを集めたいか
- クローラがWebサイトに到達したら，すぐに違うWebサイトに行くのか，それとも到達点のWebサイトを掘り下げていくのか．
- そもそもスクレイピングが禁止されていないかどうか
- 法的に問題がないか（以前，岡崎でスクレイピングで逮捕された事例がある）

以下コードは，例外処理を施していないので403エラーとかに引っかかりがちである．うまいこと例外処理と組み合わせれば，<br>403エラーに引っかかっても違うリンクを探して．．．ってできる．エラーで止まってしまうのが一番勿体無いみたい．

In [30]:
pages = set()

# ページ内の全ての内部リンクを抽出する関数
def get_internal_links(bs,include_url):
    '''
    urlparseで対象のURLを以下の6つに分ける
    http://www.cwi.nl:80/%7Eguido/Python.html
    ↓
    scheme:     'http'
    netloc:     'www.cwi.nl:80'
    path:       '/%7Eguido/Python.html'
    params:     ''
    query:      ''
    fragment:   ''
    
    '''
    include_url = '{0}://{1}'.format(
        urlparse(include_url).scheme, 
        urlparse(include_url).netloc
    )

    #内部リンクを格納するリスト
    internal_links = []
    # '/'から始まる全てのリンクを発掘する
    for link in bs.find_all('a',href=re.compile('^(/|.*'+include_url+')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internal_links:
                if link.attrs['href'].startswith('/'):
                    internal_links.append(include_url+link.attrs['href'])
                else:
                    internal_links.append(link.attrs['href'])

    return internal_links


# ページ内の全ての外部リンクのリストを取得する関数
def get_external_links(bs, exclude_url):
    external_links = []
    # 現在のURLを含まない"http"もしくは"www"から始まる全てのリンクを見つける
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!'+ exclude_url +').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in external_links:
                external_links.append(link.attrs['href'])

    return external_links

# ランダムに外部リンクを返す関数
def get_random_external_link(start_page):
    html = urlopen(start_page)
    bs = BeautifulSoup(html, 'html.parser')
    external_links = get_external_links(bs, urlparse(start_page).netloc)

    # 外部リンクが見つからんかったとき,内部リンクに行って，そこで全ての外部リンクを取得する．
    if len(external_links)==0:
        print('No external links, looking around the site for one.')
        domain = f'{urlparse(start_page).scheme}://{urlparse(start_page.netloc)}'
        internal_links = get_internal_links(bs, domain)

        return get_random_external_link(internal_links[random.randint(0,len(internal_links)-1)])
    
    # 外部リンクがあったとき
    else:
        return external_links[random.randint(0,len(external_links)-1)]


def follow_external_only(start_site):
    external_link = get_random_external_link(start_site)
    print(f'Random external link is: {external_link}')
    follow_external_only(external_link)


if __name__=='__main__':
    follow_external_only('http://oreilly.com')

Random external link is: https://play.google.com/store/apps/details?id=com.safariflow.queue
Random external link is: http://developer.android.com/index.html
Random external link is: https://play.google.com/apps/publish
Random external link is: https://medium.com/googleplaydev/experiment-with-localized-items-and-characters-822cc8f17420?source=collection_home---4------0-----------------------


HTTPError: HTTP Error 403: Forbidden

例外処理が定まりきっていないうちは，アクセスせずにリンク格納くらいに単純化しておいた方がいいとのこと．

In [33]:
all_int_links = set()
all_ext_links = set()

def get_all_external_links(site_url):
    html = urlopen(site_url)
    domain = f'{urlparse(site_url).scheme}://{urlparse(site_url).netloc}'
    bs = BeautifulSoup(html, 'html.parser')

    internal_links = get_internal_links(bs, domain)
    external_links = get_external_links(bs, domain)

    for link in external_links:
        if link not in all_ext_links:
            all_ext_links.add(link)
            print(link)

    print('----------------------------------------------------------')

    for link in internal_links:
        if link not in all_int_links:
            all_int_links.add(link)
            print(link)

if __name__ == '__main__':
    all_int_links.add('http://oreilly.com')
    get_all_external_links('http://oreilly.com')

https://www.oreilly.com
https://learning.oreilly.com/accounts/login-check/
https://www.oreilly.com/online-learning/try-now.html
https://www.oreilly.com/online-learning/teams.html
https://www.oreilly.com/online-learning/government.html
https://www.oreilly.com/online-learning/academic.html
https://www.oreilly.com/online-learning/individuals.html
https://www.oreilly.com/online-learning/features.html
https://www.oreilly.com/online-learning/feature-certification.html
https://www.oreilly.com/online-learning/intro-interactive-learning.html
https://www.oreilly.com/online-learning/live-events.html
https://www.oreilly.com/online-learning/feature-answers.html
https://www.oreilly.com/radar/
https://www.oreilly.com/content-marketing-solutions.html
https://learning.oreilly.com/p/register/
https://www.oreilly.com/online-learning/the-cost-of-doing-nothing.html
https://www.oreilly.com/online-learning/radar-event-building-workforce-2022.html
https://learning.oreilly.com/search/?query=author%3A%22Arianne